In [466]:
import mailbox
import re
import pprint as pp
from bs4 import BeautifulSoup
import pandas as pd
import csv
from textblob import TextBlob

In [561]:
from abc import ABCMeta, abstractmethod


class FeatureFinder:
    __metaclass__ = ABCMeta

    @abstractmethod
    def getFeatureTitle(self):
        pass

    @abstractmethod
    def getFeature(self, message):
        pass


class HTMLFormFinder(FeatureFinder):
    def getFeatureTitle(self):
        return "Html Form"

    def getFeature(self, message):
        import re
        super(HTMLFormFinder, self).getFeature(message)
        payload = getpayload(message).lower()
        return re.compile(r'<\s?\/?\s?form\s?>', re.IGNORECASE).search(payload) != None


class IFrameFinder(FeatureFinder):
    def getFeatureTitle(self):
        return "Html iFrame"

    def getFeature(self, message):
        import re
        super(IFrameFinder, self).getFeature(message)
        payload = getpayload(message).lower()
        return re.compile(r'<\s?\/?\s?iframe\s?>', re.IGNORECASE).search(payload) != None


class FlashFinder(FeatureFinder):
    def getFeatureTitle(self):
        return "Flash content"

    def getFeature(self, message):
        import re
        super(FlashFinder, self).getFeature(message)
        payload = getpayload(message).lower()

        swflinks = re.compile(FLASH_LINKED_CONTENT, re.IGNORECASE).findall(payload)
        flashObject = re.compile(r'embed\s*src\s*=\s*\".*\.swf\"', re.IGNORECASE).search(payload);
        return (swflinks != None and len(swflinks) > 0) or \
               (flashObject != None)


class AttachmentFinder(FeatureFinder):
    def getFeatureTitle(self):
        return "Attachments"

    def getFeature(self, message):
        return getAttachmentCount(message)


class HTMLContentFinder(FeatureFinder):
    def getFeatureTitle(self):
        return "HTML content"

    def getFeature(self, message):
        return ishtml(message)


class URLsFinder(FeatureFinder):
    def getFeatureTitle(self):
        return "URLs"

    def getFeature(self, message):
        return len(geturls_payload(message))


class ExternalResourcesFinder(FeatureFinder):
    def getFeatureTitle(self):
        return "External Resources"

    def getFeature(self, message):
        return len(getexternalresources(message))


class JavascriptFinder(FeatureFinder):
    def getFeatureTitle(self):
        return "Javascript"

    def getFeature(self, message):
        return len(getjavascriptusage(message))


class CssFinder(FeatureFinder):
    def getFeatureTitle(self):
        return "Css"

    def getFeature(self, message):
        return len(getcssusage(message))


class IPsInURLs(FeatureFinder):
    def getFeatureTitle(self):
        return "IPs in URLs"

    def getFeature(self, message):
        return len(getIPHrefs(message)) > 0


class AtInURLs(FeatureFinder):
    def getFeatureTitle(self):
        return "@ in URLs"

    def getFeature(self, message):
        emailPattern = re.compile(EMAILREGEX, re.IGNORECASE)
        for url in geturls_payload(message):
            if (url.lower().startswith("mailto:") or (
                    emailPattern.search(url) != None and emailPattern.search(url).group() != None)):
                continue
            atvalue = url.find("@")
            athexvalue = url.find("%40")

            if (atvalue != -1 and athexvalue != -1):
                atvalue = min(athexvalue, atvalue)
            else:
                atvalue = max(atvalue, athexvalue)

            paramindex = url.find("?")

            if paramindex != -1:  # url has parameters, an email can be a parameter
                if (atvalue != -1) and (paramindex > atvalue):
                    return True
            else:
                # There are no parameters in the url. if there is an @, then return true
                if (atvalue != -1):
                    return True
        return False
    
class EncodingFinder(FeatureFinder):
    def getFeatureTitle(self):
        return "Encoding"

    def getFeature(self, message):
        return str(message.get('content-transfer-encoding')).lower().strip()
    
class TextAnalysis(FeatureFinder):
    def getFeatureTitle(self):
        return "TextAnalysis"
    
    def getFeature(self, message):
        return get_text_analysis(message)

In [656]:
def getpayload(msg):
    return __getpayload_rec__(msg, payloadresult="")


def __getpayload_rec__(msg, payloadresult):
    payload = msg.get_payload()
#     print(payload)
    if str(msg.get('content-transfer-encoding')).lower() == "base64":
        
        payload = msg.get_payload(decode=True)
#         print("entered here:", payload)
#         print(type(payload))

    if payload and msg.is_multipart():
        for subMsg in payload:
            payloadresult += __getpayload_rec__(subMsg, payloadresult)
    else:
        if type(payload) == bytes:
#             print("AM BYTES")
            payload_asbytes = str(payload)
#             try:

#             except (UnicodeDecodeError, AttributeError):
#                 print("Found Error!")
#                 pass
        
            return msg.get_content_type() + "\t" + payload_asbytes + "\n"
        else:
            return msg.get_content_type() + "\t" + payload + "\n"
    return payloadresult

def getpayload_dict(msg):
    return __getpayload_dict_rec__(msg, [])


def __getpayload_dict_rec__(msg, payloadresult):
    payload = msg.get_payload()
    if msg.is_multipart():
        for subMsg in payload:
            __getpayload_dict_rec__(subMsg, payloadresult)
    else:
        payloadresult.append({"mimeType": msg.get_content_type(), "payload": payload})
    return payloadresult


def getAttachmentCount(msg):
    return __getAttachmentCountrec__(msg, count=0)


def __getAttachmentCountrec__(msg, count):
    payload = msg.get_payload()
    if msg.is_multipart():
        for subMsg in payload:
            count += __getAttachmentCountrec__(subMsg, count)
    else:
        if __hasAttachment__(msg):
            return 1
    return count


def __hasAttachment__(message):
    contentDisp = message.get("Content-Disposition")
#     print(message)
    return contentDisp is not None and contentDisp.lower().find("attachment") != -1


def getContentTypes(msg):
    return __getContentTypes_rec__(msg, [])


def __getContentTypes_rec__(msg, contenttypes):
    payload = msg.get_payload()
    if msg.is_multipart():
        for subMsg in payload:
            __getContentTypes_rec__(subMsg, contenttypes)
    else:
        contenttypes.append(msg.get_content_type())

    return contenttypes

def geturls_payload(message):
    """
    Returns the urls present in the message payload.
    Could be optimized by only looking into text payloads instead of all the payload

    :param message: message
    :return: url list
    """
    return geturls_string(getpayload(message))

def geturls_string(string):
    """
    Returns the urls present in the message payload.
    Could be optimized by only looking into text payloads instead of all the payload

    :param message: message
    :return: url list
    """
    result = []

    cleanPayload = re.sub(r'\s+', ' ', string)  # removes innecesary spaces
    linkregex = re.compile(HREFREGEX, re.IGNORECASE)
    links = linkregex.findall(cleanPayload)

    for link in links:
        if isurl(link):
            result.append(link)


    urlregex = re.compile(URLREGEX_NOT_ALONE, re.IGNORECASE)
    links = urlregex.findall(cleanPayload)
    for link in links:
        if link not in result:
            result.append(link)
    return links

def isurl(link):
    return re.compile(URLREGEX, re.IGNORECASE).search(link) is not None


def getIPHrefs(message):
    urls = geturls_payload(message)
    iphref = re.compile(IPREGEX, re.IGNORECASE)
    result = []
    for url in urls:
        if iphref.search(url) and iphref.search(url).group(1) is not None:
            result.append(iphref.search(url).group(1))
    return result


def getexternalresources(message):
    """
    :param message: message
    :return: url list-
    """
    result = []

    for script in getjavascriptusage(message):
        if "src" in str(script) and "src" in script and isurl(script["src"]):
            result.append(script["src"])
    for css in getcssusage(message):
        if "href" in str(css) and isurl(css["href"]):
            result.append(css["href"])

    return result


def getjavascriptusage(message):
    """
    :param message: message
    :return: url list-
    """
    result = []
    payload = getpayload_dict(message)
    for part in payload:
        if part["mimeType"].lower() == "text/html":
            htmlcontent = part["payload"]
            soup = BeautifulSoup(htmlcontent, "html.parser")
#             print(soup)
            scripts = soup.find_all("script")
#             print(scripts)
            for script in scripts:
                result.append(script)
    return result


def getcssusage(message):
    """
    :param message: message
    :return: url list-
    """
    result = []
    payload = getpayload_dict(message)
    
#     print("len payload: ", len(payload))
    for part in payload:
        if part["mimeType"].lower() == "text/html":
#             print("ENTERED HERE")
            htmlcontent = part["payload"]
#             print("htmlcontent", htmlcontent)
#             print("html len: ", len(htmlcontent))
            soup = BeautifulSoup(htmlcontent, "html.parser")
#             print("soup", soup)
#             print("soup len: ", len(soup))
            csslinks = soup.findAll("link")
#             print(csslinks)
            for css in csslinks:
                result.append(css)
    return result


def extract_registered_domain(url):
    return tldextract.extract(url).registered_domain

def get_whois_data(url):
    domain = extract_registered_domain;
    return pythonwhois.get_whois(domain)


def ishtml(message):
    result = ("text/html" in getContentTypes(message))
    payload = getpayload_dict(message)
#     print(getContentTypes(message))
#     print(result)
    for part in payload:
#         print("beautifulSoup: ", BeautifulSoup(part["payload"], "html.parser").find())
        if result or BeautifulSoup(part["payload"], "html.parser").find():
            return True
    return result

def get_text_analysis(some_message):
#     print("get_text_analysis: ", some_message)
    payload = getpayload_dict(some_message)
    
#     pp.pprint(payload)
    j = 0
    sentences = []
    for part in payload:
        print(j, '-' * 77)
#         pp.pprint(part)
        if part['mimeType'] == "text/html":
            print("PARSING HTML")
            htmlcontent = BeautifulSoup(part['payload'], "html.parser")
            htmltext = ''.join(htmlcontent.findAll(text=True))
#             for k in range(len(htmltext)):
#                 print(k, "*" * 40, "\n", htmltext[k])
            blob = TextBlob(htmltext)
            for k in range(len(blob.sentences)):
#                 print('\t', k, blob.sentences[k])
                sentences.append(blob.sentences[k])
#         pp.pprint(blob.sentences)
        elif part['mimeType'] == "text/plain":
#             print("PARSING PLAINTEXT")
            blob = TextBlob(part['payload'])
            for k in range(len(blob.sentences)):
#                 print('\t', k, blob.sentences[k])
                sentences.append(blob.sentences[k])
            
        else:
            print("UNPARSED TEXT OF MIMTYPE: ", part['mimeType'])
        j+=1

    return sentences

In [600]:
URLREGEX = r"^(https?|ftp)://[^\s/$.?#].[^\s]*$"
URLREGEX_NOT_ALONE = r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"
FLASH_LINKED_CONTENT = r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F])+).*\.swf"
HREFREGEX = '<a\s*href=[\'|"](.*?)[\'"].*?\s*>'
IPREGEX =  r"\b((25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?))\b"
EMAILREGEX = r"([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)"


# NLP 

In [654]:
from textblob import TextBlob
from textblob.classifiers import NaiveBayesClassifier
from textblob.classifiers import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

import pandas as pd
import time

random_state = 42

def trainNLP(filenames, stop = None, phishy = [], idxParse=None):
    #     filename = "emails-enron"
    myfinders = [TextAnalysis()]
    idxPhishy = 0
    
    parseAll = idxParse == None

    data=[]
    email_index = []
    
    train = [] 
    
    
    
    for filename in filenames:
        i = 1
        mbox = mailbox.mbox("./resources/" + filename + ".mbox")
        for msg in mbox:
            
            totalsize = 0
            payload = getpayload_dict(msg)
            for part in payload:
                totalsize += len(re.sub(r'\s+','',part["payload"]))

            if totalsize < 1:
                print(i, " empty email - ", str(phishy), " - ", payload)
                i += 1
                continue
                      
            if parseAll or (idxParse and i in idxParse):

                for finder in myfinders:
                    tmpDict[finder.getFeatureTitle()] = finder.getFeature(msg)

                tmpDict["Phishy"] = phishy
#                 tmpDict["id"] = i


                for finder in myfinders:
                    tmpDict[finder.getFeatureTitle()] = finder.getFeature(msg)

    #             pp.pprint(tmpDict)
                data.append(tmpDict)
            
            i+=1
            if stop and i >= stop:
                break
        train.extend(makeTrain(data, phishy[idxPhishy]))
        idxPhishy +=1
        
#     pp.pprint(train)
#     cl = NaiveBayesClassifier(train)
    
    return train
        
def makeTrain(data, phishy):
    if phishy == True:
        label = "pos"
    else:
        label = "neg"
        
    train = []
    for d in data:
        sentences = d["TextAnalysis"] 
#         print(type(sentences))
#         pp.pprint(sentences)
        
        for s in sentences:
            train.append((str(s.strip()), label))
            
#     pp.pprint(train)
    return train


        
start = time.perf_counter()
trainSet = trainNLP(["emails-phishing", "emails-enron"], phishy = [True, False], idxParse=None)
end = time.perf_counter()
print("Extracting sentences from emails", end-start, "sec")

Xy_train, Xy_test = train_test_split(trainSet, test_size=0.33, random_state=random_state)



0 -----------------------------------------------------------------------------
PARSING HTML
1 -----------------------------------------------------------------------------
UNPARSED TEXT OF MIMTYPE:  image/gif
0 -----------------------------------------------------------------------------
PARSING HTML
1 -----------------------------------------------------------------------------
UNPARSED TEXT OF MIMTYPE:  image/gif
0 -----------------------------------------------------------------------------
PARSING HTML
0 -----------------------------------------------------------------------------
PARSING HTML
0 -----------------------------------------------------------------------------
PARSING HTML
0 -----------------------------------------------------------------------------
PARSING HTML
0 -----------------------------------------------------------------------------
PARSING HTML
0 -----------------------------------------------------------------------------
PARSING HTML
0 --------------------

0 -----------------------------------------------------------------------------
PARSING HTML
1 -----------------------------------------------------------------------------
UNPARSED TEXT OF MIMTYPE:  image/gif
0 -----------------------------------------------------------------------------
PARSING HTML
1 -----------------------------------------------------------------------------
UNPARSED TEXT OF MIMTYPE:  image/gif
0 -----------------------------------------------------------------------------
PARSING HTML
1 -----------------------------------------------------------------------------
UNPARSED TEXT OF MIMTYPE:  application/octet-stream
0 -----------------------------------------------------------------------------
PARSING HTML
1 -----------------------------------------------------------------------------
UNPARSED TEXT OF MIMTYPE:  application/octet-stream
0 -----------------------------------------------------------------------------
PARSING HTML
1 -----------------------------------

0 -----------------------------------------------------------------------------
PARSING HTML
0 -----------------------------------------------------------------------------
PARSING HTML
0 -----------------------------------------------------------------------------
PARSING HTML
0 -----------------------------------------------------------------------------
PARSING HTML
0 -----------------------------------------------------------------------------
PARSING HTML
0 -----------------------------------------------------------------------------
PARSING HTML
0 -----------------------------------------------------------------------------
PARSING HTML
0 -----------------------------------------------------------------------------
PARSING HTML
0 -----------------------------------------------------------------------------
PARSING HTML
0 -----------------------------------------------------------------------------
PARSING HTML
0 --------------------------------------------------------------------

0 -----------------------------------------------------------------------------
PARSING HTML
0 -----------------------------------------------------------------------------
PARSING HTML
0 -----------------------------------------------------------------------------
PARSING HTML
0 -----------------------------------------------------------------------------
PARSING HTML
0 -----------------------------------------------------------------------------
PARSING HTML
0 -----------------------------------------------------------------------------
PARSING HTML
0 -----------------------------------------------------------------------------
PARSING HTML
0 -----------------------------------------------------------------------------
PARSING HTML
0 -----------------------------------------------------------------------------
PARSING HTML
0 -----------------------------------------------------------------------------
PARSING HTML
0 --------------------------------------------------------------------

0 -----------------------------------------------------------------------------
PARSING HTML
0 -----------------------------------------------------------------------------
PARSING HTML
0 -----------------------------------------------------------------------------
PARSING PLAINTEXT
1 -----------------------------------------------------------------------------
PARSING HTML
0 -----------------------------------------------------------------------------
PARSING PLAINTEXT
1 -----------------------------------------------------------------------------
PARSING HTML
0 -----------------------------------------------------------------------------
PARSING HTML
0 -----------------------------------------------------------------------------
PARSING HTML
0 -----------------------------------------------------------------------------
PARSING HTML
0 -----------------------------------------------------------------------------
PARSING HTML
0 ----------------------------------------------------------

...

0 -----------------------------------------------------------------------------
PARSING PLAINTEXT
0 -----------------------------------------------------------------------------
PARSING PLAINTEXT
0 -----------------------------------------------------------------------------
PARSING PLAINTEXT
0 -----------------------------------------------------------------------------
PARSING PLAINTEXT
0 -----------------------------------------------------------------------------
PARSING PLAINTEXT
0 -----------------------------------------------------------------------------
PARSING PLAINTEXT
0 -----------------------------------------------------------------------------
PARSING PLAINTEXT
0 -----------------------------------------------------------------------------
PARSING PLAINTEXT
0 -----------------------------------------------------------------------------
PARSING PLAINTEXT
0 -----------------------------------------------------------------------------
PARSING PLAINTEXT
0 ------------------

0 -----------------------------------------------------------------------------
PARSING PLAINTEXT
0 -----------------------------------------------------------------------------
PARSING PLAINTEXT
0 -----------------------------------------------------------------------------
PARSING PLAINTEXT
0 -----------------------------------------------------------------------------
PARSING PLAINTEXT
0 -----------------------------------------------------------------------------
PARSING PLAINTEXT
0 -----------------------------------------------------------------------------
PARSING PLAINTEXT
0 -----------------------------------------------------------------------------
PARSING PLAINTEXT
0 -----------------------------------------------------------------------------
PARSING PLAINTEXT
0 -----------------------------------------------------------------------------
PARSING PLAINTEXT
0 -----------------------------------------------------------------------------
PARSING PLAINTEXT
0 ------------------

0 -----------------------------------------------------------------------------
PARSING PLAINTEXT
0 -----------------------------------------------------------------------------
PARSING PLAINTEXT
0 -----------------------------------------------------------------------------
PARSING PLAINTEXT
0 -----------------------------------------------------------------------------
PARSING PLAINTEXT
0 -----------------------------------------------------------------------------
PARSING PLAINTEXT
0 -----------------------------------------------------------------------------
PARSING PLAINTEXT
0 -----------------------------------------------------------------------------
PARSING PLAINTEXT
0 -----------------------------------------------------------------------------
PARSING PLAINTEXT
0 -----------------------------------------------------------------------------
PARSING PLAINTEXT
0 -----------------------------------------------------------------------------
PARSING PLAINTEXT
0 ------------------

0 -----------------------------------------------------------------------------
PARSING PLAINTEXT
0 -----------------------------------------------------------------------------
PARSING PLAINTEXT
0 -----------------------------------------------------------------------------
PARSING PLAINTEXT
0 -----------------------------------------------------------------------------
PARSING PLAINTEXT
0 -----------------------------------------------------------------------------
PARSING PLAINTEXT
0 -----------------------------------------------------------------------------
PARSING PLAINTEXT
0 -----------------------------------------------------------------------------
PARSING PLAINTEXT
0 -----------------------------------------------------------------------------
PARSING PLAINTEXT
0 -----------------------------------------------------------------------------
PARSING PLAINTEXT
0 -----------------------------------------------------------------------------
PARSING PLAINTEXT
0 ------------------

Extracting sentences from emails 46.100097858012305 sec


In [602]:
print(len(Xy_train))
print(len(Xy_test))
# pp.pprint(Xy_train)

20849
10269


In [603]:
import time 
start = time.perf_counter()
nbcl = NaiveBayesClassifier(Xy_train)
end = time.perf_counter()
print("Training NaiveBayesClassifier with", len(Xy_train), " training points took", end-start, "seconds")


Training NaiveBayesClassifier with 20849  training points took 32.336588085992844 seconds


In [625]:
nbcl_acc = nbcl.accuracy(Xy_test)
print("Accuracy of NaiveBayesClassifier: ", round(nbcl_acc, 10))

Accuracy of NaiveBayesClassifier:  1.0


In [626]:
nbcl.informative_features()

[('contains(on)', True),
 ('contains(your)', False),
 ('contains(JAVELIN.MPG)', False),
 ('contains(BRONZE.MPG)', False),
 ('contains(attached)', False),
 ('contains(See)', False),
 ('contains(GOLD.MPG)', False),
 ('contains(SILVER.MPG)', False),
 ('contains(file)', False),
 ('contains(the)', True),
 ('contains(of)', True),
 ('contains(records)', False),
 ('contains(talgar)', False),
 ('contains(ss.ps.ge.com)', False),
 ('contains(Forwarded)', False),
 ('contains(09/22/2000)', False),
 ('contains(lcoleman)', False),
 ('contains(sscott5.nsf)', False),
 ('contains(Susan_Scott_Dec2000_June2001_2\\Notes)', False),
 ('contains(To)', False),
 ('contains(Jackson)', False),
 ('contains(Don)', False),
 ('contains(Perkins)', False),
 ('contains(BigTex)', False),
 ('contains(Olympics)', False),
 ('contains(X-cc)', False),
 ('contains(AAllett)', False),
 ('contains(castledental.com)', False),
 ('contains(X-From)', False),
 ('contains(Angela)', False),
 ('contains(X-FileName)', False),
 ('contains(

In [627]:
import time 
start = time.perf_counter()
dtcl = DecisionTreeClassifier(Xy_train)
end = time.perf_counter()
print("Training DecisionTreeClassifier with", len(Xy_train), " training points took", end-start, "seconds")


Training DecisionTreeClassifier with 20849  training points took 34.30724886400276 seconds


In [635]:
from  sklearn.metrics import classification_report

y_true = [x[1] for x in Xy_test]

# X_test = [x[0] for x in Xy_test]
# print(X_test)

y_pred = []
for d in Xy_test:
    y_pred.append(dtcl.classify(d[0]))
    
# pp.pprint(y_true)
# print(X)
# print(y_pred)
print(classification_report(y_true, y_pred))

# dtcl_acc = dtcl.accuracy(X_test)

print("Accuracy of DecisionTreeClassifier: ", round(dtcl_acc, 10))

              precision    recall  f1-score   support

         neg       1.00      1.00      1.00      4296
         pos       1.00      1.00      1.00      5973

    accuracy                           1.00     10269
   macro avg       1.00      1.00      1.00     10269
weighted avg       1.00      1.00      1.00     10269

Accuracy of DecisionTreeClassifier:  1.0


In [611]:
dtcl.pprint()

'contains(your)=False? ................................. neg\n  contains(pass-through)=False? ....................... neg\n    contains(rights)=False? ........................... neg\n    contains(rights)=True? ............................ pos\n  contains(pass-through)=True? ........................ pos\ncontains(your)=True? .................................. pos\n'

In [636]:
from joblib import dump, load
model_filename = "DTTextClassifier.dat"
dump(dtcl, model_filename)

model_filename = "NBTextClassifier.dat"
dump(nbcl, model_filename)


['DTTextClassifier.dat']

In [659]:
from collections import defaultdict            


def parseFile(filename, stop = None, phishy = True, idxParse=None):
#     filename = "emails-enron"
    mbox = mailbox.mbox("./resources/" + filename + ".mbox")
    i = 1
    s = set()
    d = defaultdict(list)

    finders = [HTMLFormFinder(), AttachmentFinder(), FlashFinder(),
               IFrameFinder(), HTMLContentFinder(), URLsFinder(),
               ExternalResourcesFinder(), JavascriptFinder(),
               CssFinder(), IPsInURLs(), AtInURLs(), EncodingFinder()]
    
    myfinders = [TextAnalysis()]

    parseAll = idxParse == None

    data=[]
    email_index = []
    

    for msg in mbox:
    #     print("msg type: ", type(msg))
    #     print(i)
    #     payload = getpayload_dict(msg)
        tmpDict = {}

    #     print(payload)

    #         print(item.keys())
    #         print(item["mimeType"])

    #     pp.pprint(payload)

    #     attachmentCount = getAttachmentCount(msg)
    #     contentTypes = getContentTypes(msg)
    #     urls = geturls_payload(msg)
    #     iphRefs = getIPHrefs(msg)
    #     externalResources = getexternalresources(msg)

        totalsize = 0
        payload = getpayload_dict(msg)
        for part in payload:
            totalsize += len(re.sub(r'\s+','',part["payload"]))

        if totalsize < 1:
            print(i, " empty email - ", str(phishy), " - ", payload)
            i += 1
            continue

    #     print(i, " : ", msg['content-transfer-encoding'])
    #     for item in payload:
    #         s.add(item["mimeType"])
    #         d[item["mimeType"]].append(i)
        
        if parseAll or (idxParse and i in idxParse):

            for finder in finders:
                tmpDict[finder.getFeatureTitle()] = finder.getFeature(msg)

            tmpDict["Phishy"] = phishy
#             tmpDict["id"] = i
            
            
#             for finder in myfinders:
#                 tmpDict[finder.getFeatureTitle()] = finder.getFeature(msg)

#             pp.pprint(tmpDict)
            data.append(tmpDict)
            email_fields = {}

            email_fields["id"] = i
            email_fields["message"] = getpayload(msg)


    #         try:
    #             print(msg)
    #             email_fields["raw"] = str(msg)

    #         except UnicodeEncodeError as er:
    #             print("type: ", type(msg))
    #             email_fields["raw"] = msg.as_bytes()
    #             print(er)
        #         print(msg.get_body_encoding())
    #             print(msg.get_charsets())
    #         except: 
    #             print("CRITICAL ERROR")

            email_index.append(email_fields)

        if iphRefs:
            print(iphRefs)

        if len(externalResources):
            print(externalResources)


        #fixme
        i += 1
        if stop and i >= stop:
            break


#     pp.pprint(data)
    df = pd.DataFrame(data)

    df.to_csv("./datasets/" + filename + "-export.csv", quoting=csv.QUOTE_ALL)



    emails = pd.DataFrame(email_index)
    emails.to_csv("./datasets/" + filename + "-export-raw.csv")

    # print(d)
    for k,v in d.items():
        print("{0:30} : {1}".format(k, len(v)))
    print(s)
    display(df.tail())
    

parseFile("emails-enron", phishy=False, idxParse = None)
parseFile("emails-phishing", phishy=True, idxParse = None)

set()


,@ in URLs,Attachments,Css,Encoding,External Resources,Flash content,HTML content,Html Form,Html iFrame,IPs in URLs,Javascript,Phishy,URLs
4274,False,0,0,7bit,0,False,False,False,False,False,0,False,0
4275,False,0,0,7bit,0,False,False,False,False,False,0,False,1
4276,False,0,0,7bit,0,False,True,False,False,False,0,False,0
4277,False,0,0,7bit,0,False,False,False,False,False,0,False,0
4278,False,0,0,none,0,False,True,False,False,False,0,False,0


14  empty email -  True  -  [{'mimeType': 'text/plain', 'payload': ''}]
316  empty email -  True  -  [{'mimeType': 'text/plain', 'payload': ''}]
723  empty email -  True  -  [{'mimeType': 'text/plain', 'payload': ''}]
807  empty email -  True  -  [{'mimeType': 'text/plain', 'payload': ''}]
829  empty email -  True  -  [{'mimeType': 'text/plain', 'payload': ''}]
858  empty email -  True  -  [{'mimeType': 'text/plain', 'payload': ''}]
863  empty email -  True  -  [{'mimeType': 'text/plain', 'payload': ''}]
864  empty email -  True  -  [{'mimeType': 'text/plain', 'payload': ''}]
865  empty email -  True  -  [{'mimeType': 'text/plain', 'payload': ''}]
985  empty email -  True  -  [{'mimeType': 'text/plain', 'payload': ''}]
1111  empty email -  True  -  [{'mimeType': 'text/plain', 'payload': ''}]
1213  empty email -  True  -  [{'mimeType': 'text/plain', 'payload': ''}]
1328  empty email -  True  -  [{'mimeType': 'text/plain', 'payload': ''}]
1408  empty email -  True  -  [{'mimeType': 'text

,@ in URLs,Attachments,Css,Encoding,External Resources,Flash content,HTML content,Html Form,Html iFrame,IPs in URLs,Javascript,Phishy,URLs
2251,False,0,0,7bit,0,False,True,False,False,False,0,True,2
2252,False,0,2,8bit\r\n,0,False,True,False,False,False,11,True,32
2253,False,0,0,none,0,False,True,False,False,False,0,True,2
2254,False,0,0,7bit,0,False,True,False,False,False,0,True,2
2255,False,0,0,7bit,0,False,True,False,False,False,0,True,20


In [709]:
import time
from itertools import chain
import email
import imaplib
import pprint as pp
import email
import email.mime.message
import base64

import pickle

model_filename = "RandomForestModel.dat"
rf_tuned_saved = load(model_filename)

encoding = "encoding.bin"
dEncoding = load(encoding)


# import mailparser

# mail = mailparser.parse_from_file(f)
# mail = mailparser.parse_from_file_obj(fp)
# mail = mailparser.parse_from_string(raw_mail)
# mail = mailparser.parse_from_bytes(byte_mail)

import argparse


# get commandline arguments
# parser = argparse.ArgumentParser()
# parser.add_argument('-p', '--password')
# parser.add_argument('-u', '--username')
# args = parser.parse_args()

# print("Command line arguments:");print(args);print(78*'-')


imap_ssl_host = 'imap.gmail.com'  # imap.mail.yahoo.com
imap_ssl_port = 993
username = 'phishingdetective@gmail.com'
password = 'phishtooldetect97'

# Restrict mail search. Be very specific.
# Machine should be very selective to receive messages.
criteria = {
  # 'FROM':    'isaacpitblado@tamu.edu',
  # 'SUBJECT': 'PHISHING',
  # 'BODY':    'SECRET SIGNATURE',
}
uid_max = 0


def search_string(uid_max, criteria):
  c = list(map(lambda t: (t[0], '"'+str(t[1])+'"'), criteria.items())) + [('UID', '%d:*' % (uid_max+1))]
  return '(%s)' % ' '.join(chain(*c))
  # Produce search string in IMAP format:
  #   e.g. (FROM "me@gmail.com" SUBJECT "abcde" BODY "123456789" UID 9999:*)


def get_first_text_block(msg):
  type = msg.get_content_maintype()

  if type == 'multipart':
    for part in msg.get_payload():
      if part.get_content_maintype() == 'text':
        return part.get_payload()
  elif type == 'text':
    return msg.get_payload()


server = imaplib.IMAP4_SSL(imap_ssl_host, imap_ssl_port)
server.login(username, password)
server.select('INBOX')

# result, data = server.uid('search', None, search_string(uid_max, criteria))
result, data = server.search(None, 'ALL')

uids = [int(s) for s in data[0].split()]
# print("original uids: ", uids)
if uids:
#   uid_max = max(uids) # FIXME

    uid_max = 8
    print("CURRENTLY HARD CODING UID MAX FIX THIS: ", uid_max)
  # Initialize `uid_max`. Any UID less than or equal to `uid_max` will be ignored subsequently.

server.logout()

finders = [HTMLFormFinder(), AttachmentFinder(), FlashFinder(),
           IFrameFinder(), HTMLContentFinder(), URLsFinder(),
           ExternalResourcesFinder(), JavascriptFinder(),
           CssFinder(), IPsInURLs(), AtInURLs(), EncodingFinder()]


# Keep checking messages ...
# I don't like using IDLE because Yahoo does not support it.
while 1:
    # Have to login/logout each time because that's the only way to get fresh results.

    server = imaplib.IMAP4_SSL(imap_ssl_host, imap_ssl_port)
    server.login(username, password)
    server.select('INBOX')

    # result, data = server.uid('search', None, search_string(uid_max, criteria))
    result, data = server.search(None, 'ALL')

    uids = [int(s) for s in data[0].split()]
    print('UID: ', uids)
    for uid in uids:
    #   for i in range(len(uids) -1, len(uids) - 2, -1):
    # Have to check again because Gmail sometimes does not obey UID criterion.
    #     uid = uids[i]
        if uid > uid_max:
        #     if True:
            print("uid: ", uid)
            result, data = server.uid('fetch', str(uid), '(RFC822)')  # fetch entire message
            
            # print("res: ", result)
            # print("data: ", data)
            # print("data:")
            # pp.pprint(data)

            print("setting new uid_max: ", uid)
            uid_max = uid
            
#             print("data:", data)
            phishy_sender = ""
            
            attached_emails = []
        
            results = [] 

            for response_part in data:
                if isinstance(response_part, tuple):
                    # print("response_part: ", response_part)
                    # print("response_part[1]: ", response_part[1])
                    msg = email.message_from_string(response_part[1].decode('utf-8'))
#                     msg = email.message_from_string(response_part[1].decode('utf-8'))
#                     print(msg)
                    phishy_sender = msg["From"]
                    print("from: ", msg["From"])
                    

                    for part in msg.walk():
                        curr_email = {}
                        if part['Content-Type'] and part['Content-Type'].find("message/rfc822") != -1:

                            attachment = part.get_payload()
#                             print(part.keys())
#                             print(part.values())
#                             print(part.get_filename())
#                             curr_email["filename"] = part.get_filename()

#                             print("PAYLOAD LEN: ", len(attachment))
                            for i in range(len(attachment)):
#                                 print("payload i: ",i, attachment[i].as_string())
#                                 print("keys: ", part[i].keys())
#                                 print("type: ", type(attachment[i]))
                                decoded = email.message_from_bytes(base64.b64decode(attachment[i].as_string()))
                                attached_message = getpayload_dict(decoded)
                                print(attached_message)
                                totalsize = 0
                                for a_part in attached_message:
                                    totalsize += len(re.sub(r'\s+','',a_part["payload"]))

                                if totalsize < 1:
                                    print(" empty email - ", attached_message)
                                    i += 1
                                    continue
                                
                                tmpDict = {}
                                for finder in finders:
                                    tmpDict[finder.getFeatureTitle()] = finder.getFeature(decoded)
                                
                                tmpDict['filename'] = part.get_filename()

                                pp.pprint(tmpDict)
                                attached_emails.append(tmpDict)
                            
            
                                
                        else:
                            print("not message/rfc822: ", part['Content-Type'])
                        
                    
                    X = pd.DataFrame(attached_emails)
                    
                    X = X.replace({"Encoding": dEncoding})  
                    
                    files = X['filename'].values
                    X = X.drop(columns='filename')
                    display(X.head())
                

                    prediction = rf_tuned_saved.predict(X)
                    prob = rf_tuned_saved.predict_proba(X)
                    classes = ["'Not Phishy'", "'Phishy'"]
                    print("prediction:", files, prediction)
                    
                    assert(len(files) == len(prediction))
                    
                    for i in range(len(files)):
                        results.append({"filename": files[i], "prediction": prediction[i], "prob": log_prob[i], "classes": classes})
                    
                    
            
            txt = constructResponseText(results)
            
            print("Response Text:\n", txt)
            sendResponse(phishy_sender, txt)
                    
                    
                    
                    
#   server.logout()
#   break

    time.sleep(1)


CURRENTLY HARD CODING UID MAX FIX THIS:  8
UID:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
uid:  9
setting new uid_max:  9
from:  Isaac Pitblado <isaacpitblado@gmail.com>
not message/rfc822:  multipart/mixed; boundary="00000000000009356f0598ade928"
not message/rfc822:  multipart/alternative; boundary="00000000000009356c0598ade926"
not message/rfc822:  text/plain; charset="UTF-8"
not message/rfc822:  text/html; charset="UTF-8"
[{'mimeType': 'text/plain', 'payload': ' Please do not reply to this email. This mailbox is not monitored and you\r\nwill not receive a response.\r\n\r\n\r\n\r\n\r\n\r\n*Hello  isaacpitblado@gmail.com <isaacpitblado@gmail.com>  You have an\r\ninstant payment of $400.00USD from Kevin Goodman\r\n(kevin_goodman207@hotmail.com <kevin_goodman207@hotmail.com>)*\r\n\r\n*You are require to ship out the item and get back to us with the tracking\r\nnumber or custom form number to our email agent@accountant.com\r\n<agent@accountant.com>  Payment Sent (Unique Transaction ID\r\n#0003817

,@ in URLs,Attachments,Css,Encoding,External Resources,Flash content,HTML content,Html Form,Html iFrame,IPs in URLs,Javascript,URLs
0,False,0,0,0,0,False,True,False,False,False,0,12


prediction: ['actual.eml'] [ True]
Response Text:
 Howdy 
I have analyzed 1 eml file(s). Here are my predictions

The file 'actual.eml' is phishy with probabilities of -1.4898310923085307 for class 'Not Phishy' and -0.25542235905326366 for class 'Phishy'

uid:  10
setting new uid_max:  10
from:  Isaac Pitblado <isaacpitblado@gmail.com>
not message/rfc822:  multipart/mixed; boundary="0000000000006e9c210598aff21e"
not message/rfc822:  multipart/alternative; boundary="0000000000006e9c1f0598aff21c"
not message/rfc822:  text/plain; charset="UTF-8"
not message/rfc822:  text/html; charset="UTF-8"
[{'mimeType': 'text/plain', 'payload': ' Please do not reply to this email. This mailbox is not monitored and you\r\nwill not receive a response.\r\n\r\n\r\n\r\n\r\n\r\n*Hello  isaacpitblado@gmail.com <isaacpitblado@gmail.com>  You have an\r\ninstant payment of $400.00USD from Kevin Goodman\r\n(kevin_goodman207@hotmail.com <kevin_goodman207@hotmail.com>)*\r\n\r\n*You are require to ship out the item 

,@ in URLs,Attachments,Css,Encoding,External Resources,Flash content,HTML content,Html Form,Html iFrame,IPs in URLs,Javascript,URLs
0,False,0,0,0,0,False,True,False,False,False,0,12
1,False,0,0,0,0,False,True,False,False,False,0,0
2,False,0,0,7,0,False,True,False,False,False,0,10


prediction: ['actual.eml' 'Phishing.eml' 'paypal.eml'] [ True False  True]
Response Text:
 Howdy 
I have analyzed 3 eml file(s). Here are my predictions

The file 'actual.eml' is phishy with probabilities of -1.4898310923085307 for class 'Not Phishy' and -0.25542235905326366 for class 'Phishy'
The file 'Phishing.eml' is not phishy with probabilities of -0.04139942452333341 for class 'Not Phishy' and -3.205116598916939 for class 'Phishy'
The file 'paypal.eml' is phishy with probabilities of -2.3405214947476827 for class 'Not Phishy' and -0.10123284286482621 for class 'Phishy'

UID:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
UID:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
UID:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
UID:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
UID:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
UID:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
UID:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
UID:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
UID:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
UID:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
UID:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

KeyboardInterrupt: 

In [707]:
import os
import smtplib
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart



def sendResponse(target, response_text):
    
    smtp_ssl_host = 'smtp.gmail.com'  # smtp.mail.yahoo.com
    smtp_ssl_port = 465
    smtp_username = 'phishingdetective@gmail.com'
    smtp_password = 'phishtooldetect97'
    smtp_sender = 'phishingdetective@gmail.com'
    
    msg = MIMEMultipart()
    msg['Subject'] = "Phishing Detector Results"
    msg['From'] = smtp_sender
    msg['To'] = target

    txt = MIMEText(response_text)
    msg.attach(txt)
    
    
    smtp_server = smtplib.SMTP_SSL(smtp_ssl_host, smtp_ssl_port)
    smtp_server.login(username, password)
    smtp_server.sendmail(smtp_sender, target, msg.as_string())
    smtp_server.quit()
    
def constructResponseText(results):
    txt = "Howdy \r\nI have analyzed " + str(len(results)) + " eml file(s). Here are my predictions\r\n\r\n"

    for res in results:
        txt += "The file '" + res['filename'] + "'"
        txt = txt + " is phishy " if res['prediction'] else txt + " is not phishy "
        txt += "with probabilities of " 
        txt += str(res["prob"][0]) + " for class " + str(res["classes"][0]) + " and " + str(res["prob"][1]) + " for class " + str(res["classes"][1])
        txt += "\r\n"

    return txt